[View in Colaboratory](https://colab.research.google.com/github/kabrol98/PASSNYC-Data-Challenge/blob/master/kaggle_challenge_05.ipynb)

## Data

Going to import all libraries and upload all necessary files here.

Uploading into colab. Setting up a virtual environment for myself is not an andeavor I'm interested in wasting this weekend on.

In [0]:
import pandas as pd
import sys
import tensorflow as tf
import numpy as np
import keras
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import *
from keras.models import load_model
import io
from google.colab import files
import math
import copy

In [18]:

uploaded = files.upload()

school_data = pd.read_csv(io.StringIO(uploaded['2016 School Explorer.csv'].decode('utf-8')))
# turnout_data = pd.read_csv(io.StringIO(uploaded['D5 SHSAT Registrations and Testers.csv'].decode('utf-8')))
nytdf = pd.read_csv(io.StringIO(uploaded['nytdf.csv'].decode('utf-8')))
middleschoolDirectory = pd.read_csv(io.StringIO(uploaded["2018_DOE_Middle_School_Directory.csv"].decode('utf-8')))


Saving 2016 School Explorer.csv to 2016 School Explorer (1).csv
Saving 2016-2017_Monthly_Attendance.csv to 2016-2017_Monthly_Attendance (1).csv
Saving 2018_DOE_Middle_School_Directory.csv to 2018_DOE_Middle_School_Directory (1).csv
Saving D5 SHSAT Registrations and Testers.csv to D5 SHSAT Registrations and Testers (1).csv
Saving nytdf.csv to nytdf (1).csv


In [19]:
middleschoolDirectory.describe()

,district,geapps_prog1,swdapps_prog1,geappsperseat_prog1,swdappsperseat_prog1,swdseats_prog1,geseats_prog1,priority5_prog1,priority6_prog1,priority7_prog1,...,surveysafety,totalstudents,Postcode,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL
count,482.000000,409.000000,409.000000,407.000000,402.000000,418.000000,356.000000,0.0,0.0,0.0,...,216.000000,218.000000,481.000000,481.000000,481.000000,481.000000,481.000000,481.000000,4.790000e+02,4.790000e+02
mean,15.661826,391.691932,72.911980,5.987715,5.004975,18.236842,76.264045,NaN,NaN,NaN,...,83.759259,463.908257,10792.120582,40.739306,-73.913777,7.877339,24.006237,8207.266112,2.728300e+06,2.652594e+09
std,9.371061,370.050901,49.346336,6.898800,5.050861,14.655398,51.671530,NaN,NaN,NaN,...,8.709119,443.787782,542.965137,0.087753,0.076714,4.542797,14.131892,22543.774242,1.190552e+06,1.147758e+09
min,1.000000,33.000000,2.000000,1.000000,0.000000,0.000000,4.000000,NaN,NaN,NaN,...,59.000000,38.000000,10001.000000,40.508284,-74.243428,1.000000,1.000000,1.000000,1.000811e+06,1.000160e+09
25%,8.000000,176.000000,41.000000,3.000000,3.000000,11.000000,49.000000,NaN,NaN,NaN,...,79.000000,192.750000,10451.000000,40.670986,-73.956716,4.000000,12.000000,175.000000,2.005660e+06,2.026365e+09
50%,14.500000,291.000000,62.000000,4.000000,4.000000,16.000000,69.000000,NaN,NaN,NaN,...,85.000000,266.000000,11102.000000,40.733514,-73.916655,8.000000,22.000000,369.000000,3.044493e+06,3.013670e+09
75%,24.000000,481.000000,91.000000,7.000000,5.750000,22.000000,89.250000,NaN,NaN,NaN,...,90.000000,456.750000,11231.000000,40.822443,-73.875183,12.000000,36.000000,956.000000,3.413796e+06,3.084900e+09
max,32.000000,4183.000000,432.000000,80.000000,58.000000,204.000000,461.000000,NaN,NaN,NaN,...,100.000000,2175.000000,11694.000000,40.899295,-73.712965,18.000000,51.000000,157902.000000,5.149609e+06,5.078940e+09


Data loaded from the two datasets provided and the NYT SHSAT article

For some reason, ther index schooldbn is not workiong on the mioddle school directory dataset.

In [0]:
# schoolExplorer = pd.read_csv("kaggle-data/2016 School Explorer.csv", encoding = 'utf-8')
# registrationTesters = pd.read_csv("kaggle-data/D5 SHSAT Registrations and Testers.csv", encoding= 'utf-8')
# nytdf = pd.read_csv("kaggle-data/nytdf.csv", encoding = 'utf-8')
# highschoolDirectory = pd.read_csv("kaggle-data/2017-doe-high-school-directory.csv",encoding = 'utf-8')
# middleschoolDirectory = pd.read_csv("kaggle-data/2018_DOE_Middle_School_Directory.csv",encoding = 'utf-8')

schoolExplorer = school_data


# highschoolDirectory = highschoolDirectory.set_index('dbn')
middleschoolDirectory = middleschoolDirectory.set_index('schooldbn')


schoolExplorer = schoolExplorer.set_index('Location Code')
nytdf = nytdf.set_index('DBN')

explorer_nyt = schoolExplorer.join(nytdf)
nytCombined = nytdf.join(schoolExplorer)

# nytCombined.to_csv("kaggle-data/combined.csv", index=False)
nytCombined.describe()
testErrorArray = []

## Feature Engineering

Picking, amending, and converting the necessary features into usable values for a predictive model.

In [21]:
#Pick features
features = ['Percent of Students Chronically Absent','Average Math Proficiency','Economic Need Index','Average ELA Proficiency', 'Latitude', 'Longitude', 'NumSHSATTestTakers', 'Student Attendance Rate']
target = 'NumSHSATTestTakers'

nytCombinedFiltered = nytCombined.query('NumSHSATTestTakers != 0')

nytCombinedFiltered[features].describe()

,Average Math Proficiency,Economic Need Index,Average ELA Proficiency,Latitude,Longitude,NumSHSATTestTakers
count,530.000000,531.000000,530.000000,536.000000,536.000000,537.000000
mean,2.668849,0.657750,2.574906,40.738840,-73.915875,47.204842
std,0.486156,0.191868,0.375443,0.086535,0.075129,61.158551
min,1.890000,0.059000,1.910000,40.507803,-74.243221,6.000000
25%,2.270000,0.545500,2.280000,40.671637,-73.955888,15.000000
50%,2.610000,0.710000,2.490000,40.728875,-73.919367,26.000000
75%,2.980000,0.806500,2.790000,40.820391,-73.880919,46.000000
max,4.190000,0.938000,3.930000,40.899321,-73.713022,394.000000


### Helper functions
Set of functions to normalize data and remove percentages.

In [0]:
# convert string valued percentages into floats within [0,1]
def removePercent(df,columnNames):
  for i in range(len(columnNames)):
    col_string = df[columnNames[i]].str
    df[columnNames[i]] = col_string.rstrip('%').astype('float') / 100.0
    pass
  return df
  
# for values in arbitrary ranges, normalize across [-1,1]
def normalize_series(df, labels):
  for label in labels:
#     for sigma = 1 standard deviation, ~97% of datapoints should be within [-3sigma,sigma]
    df[label] = (df[label] - df[label].mean()) / (3 * df[label].std())
    pass
  return df


In [23]:
# Applying helpers to dataset

nytApplyFilters = removePercent(nytCombinedFiltered.copy(), ['Student Attendance Rate', 'Percent of Students Chronically Absent'])

nytApplyFilters = normalize_series(nytApplyFilters, ['Average Math Proficiency', 'Average ELA Proficiency'])

nytApplyFilters = nytApplyFilters.dropna(subset=[target])
nytApplyFilters.fillna(0,inplace = True)

nytApplyFilters[features].describe()


,Percent of Students Chronically Absent,Average Math Proficiency,Economic Need Index,Average ELA Proficiency,Latitude,Longitude,NumSHSATTestTakers,Student Attendance Rate
count,537.000000,5.370000e+02,537.000000,5.370000e+02,537.000000,537.000000,537.000000,537.000000
mean,0.196108,7.939025e-17,0.650400,-2.514025e-16,40.662977,-73.778229,47.204842,0.913762
std,0.150459,3.311496e-01,0.202953,3.311496e-01,1.760135,3.190588,61.158551,0.151724
min,0.000000,-5.340184e-01,0.000000,-5.903301e-01,0.000000,-74.243221,6.000000,0.000000
25%,0.100000,-2.666146e-01,0.538000,-2.618291e-01,40.670886,-73.955847,15.000000,0.920000
50%,0.170000,-3.349339e-02,0.708000,-7.538267e-02,40.728596,-73.919072,26.000000,0.940000
75%,0.270000,2.133409e-01,0.806000,1.820910e-01,40.820192,-73.880280,46.000000,0.960000
max,1.000000,1.042978e+00,0.938000,1.203107e+00,40.899321,0.000000,394.000000,1.000000


In [0]:
# Setting up features as a vector
# Separating training and testing for k-fold vallidation
num_folds = 10
N = nytApplyFilters.shape[0]

test_set = nytApplyFilters[0: N / num_folds]
train_set = nytApplyFilters[N / num_folds : N]

test_set = test_set.filter(features)
train_set = train_set.filter(features)

In [25]:
# X and Y from train sets

X = train_set.drop(target, axis=1).values

# Using the output as a percentage

Y = train_set[[target]].values

print(X.shape)
print(Y.shape)

(484, 7)
(484, 1)


## Training Neural Network


In [11]:
numFeatures = len(features)
# Define the model
model = Sequential()
activationFuntion = 'relu'
model.add(Dense(20, input_dim=numFeatures-1, activation='relu'))
# model.add(Dense(50, activation='relu'))
model.add(Dense(20, activation='relu'))
# model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
# model.compile(loss="mean_squared_error", optimizer="adam")

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','accuracy'])


#Train
model.fit(
    X,
    Y,
    epochs=200,
    shuffle=True,
    verbose=0
)


X_test = test_set.drop(target, axis=1).values
Y_test = test_set[[target]].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)

testErrorArray.append(test_error_rate)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))
print("Here: \n%s: %.8f%%" % (model.metrics_names[2], test_error_rate[2]))

The mean squared error (MSE) for the test data set is: [28634.947302476416, 142.72677036501327, 0.0]
Here: 
acc: 0.00000000%


Best "acc" achieved by this prediction network: 0.01887%.
Best MSE on real valued Num of SHSAT Test Takers: 142.65

## Radial Basis Function (RBF) Model

Going to try a Radial Basis Function Model to minimize in-sample MSE.

Value to beat: 142.65.

### Process

#### Step 1:
K-means clustering on feature sets.

#### Step 2:
Define the RBF equation as follows.

\begin{equation}
\Large
y = \sum_{k=1}^K w_ke^{-\lambda \| \mathbf{x} - \mathbf{\mu_k} \| ^ 2}
\end{equation}

with precision rate $\lambda$ where $\mu_k$ is a mean point for all K- means.

#### Step 3:
Build model to find least square values for all weights $w_k$.

#### Step 4:
Test model with different precision rates, try to minimize cost.

### Step 1: K means clustering

#### Lloyd's algorithm
I'm going to implement K-means clustering using Lloyd's algorithm on the feature vector $ \mathbf{x} $,
using the following iterative implementation for Lloyd's algotihm.

$ \mathbf{\mu_0} = random \in {\mathbf{x}}$

$ \mathbf{S_0} =  \{ \mathbf{x} : \forall \mu, \| \mathbf{x} - \mathbf{\mu_0} \| \leq \| \mathbf{x} - \mathbf{\mu} \|\} $

$ \mathbf{\mu_{n+1}} = \frac{1}{\mathbf{\|S_n\|}} \sum_{\mathbf{x} \in \mathbf{S_n}} \mathbf{x}$

$ \mathbf{S_{n+1}} =  \{ \mathbf{x} : \forall \mu, \| \mathbf{x} - \mathbf{\mu_{n+1}} \| \leq \| \mathbf{x} - \mathbf{\mu} \|\} $


In [0]:
#  Lloyd's algorithm: Implmentation.
# Assume feature engineering has been done.
class LloydMeans:
  
  def __init__ (self, k, df, labels):
    self.pointMatrix = df[labels].as_matrix()
    self.df = df
    self.labels = labels
    self.k = k
    pass
  
  def assign_means(self, num_trials, num_iterations):
    best_error = float("inf") # Best error is positive infinity
    best_mu = []
    best_meanSet = []
    clustered_features = self.labels
    pointMatrix = self.pointMatrix
    k = self.k
    df = self.df

    for trial in range(num_trials):
      mu = self.init_mu(df, clustered_features, k)

      for i in range(num_iterations):
        #iteratively update the clusters and cluster means nunm_iterations times
        meanSet = self.update_clusters(pointMatrix, mu, k)
        mu = self.update_cluster_points(meanSet, mu, k)
        pass

      #calculate error of current mu model
      model_error = self.calculate_total_error(meanSet, mu)

      print("MSE for trial {} : {}".format(trial, model_error))

      #always take the best model w/ minimum error
      if(model_error < best_error):
        best_error = model_error
        best_mu = mu
        best_meanSet = meanSet
        pass
      pass
    print("Best error is {}".format(best_error))
#     df = assign_final_clusters(pointMatrix, best_mu, k, self.df)
    return best_mu
  
  def normalize_matrix(self, df, labels):
    mat = df[labels].as_matrix()
    # transform all datapoints so ~97% of values are in [-1, 1]
    for i in range(mat.shape[0]):
      for j in range(mat.shape[1]):
        mat[i][j] = (mat[i][j] - df[labels[j]].mean()) / (3 * df[labels[j]].std())
        pass
      pass
    return mat

  def init_mu(self, df, labels, k):
      mu = np.zeros((k, len(labels)))
      for i in range(len(labels)):
        for j in range(k):
#           initialize random float in [-1, 1]
          offset = np.random.ranf() * 6 - 3
          mu[j][i] = df[labels[i]].mean() + offset
      return mu

  def update_clusters(self, pointMatrix, mu, k):
    meanSet = [[] for i in range(k)]
  #   iterate over points
    for i in range(pointMatrix.shape[0]):
      minIndex = 0
      minDistance = np.linalg.norm(pointMatrix[i] - mu[minIndex])
  #     iterate over mu (mean points)
      for j in range(k):
        dist = np.linalg.norm(pointMatrix[i] - mu[j])
  #       pick j with the minimum distance from i
        if(dist < minDistance):
          minDistance = dist
          minIndex = j
          pass
        pass
    #   Add point i to mu[j]'s cluster'
      meanSet[minIndex].append(pointMatrix[i])
      pass
    return meanSet

  def assign_final_clusters(self, pointMatrix, mu, k, df):
  #   iterate over points (repeating update_clusters)
    for i in range(pointMatrix.shape[0]):
      minIndex = 0
      minDistance = np.linalg.norm(pointMatrix[i] - mu[minIndex])
  #     iterate over mu (mean points)
      for j in range(k):
        dist = np.linalg.norm(pointMatrix[i] - mu[j])
  #       pick j with the minimum distance from i
        if(dist < minDistance):
          minDistance = dist
          minIndex = j
          pass
        pass
    #   Assign final cluster values to dataframe
      df.loc[df.index[i], 'cluster'] = minIndex
      pass
    return df

  def update_cluster_points(self, meanSet, mu, k):
  #   iterate over mu
    for i in range(k):
      set_sum = np.zeros(mu[i].shape)
  #     iterate over mu[i]'s cluster'
      for j in range(len(meanSet[i])):
        # sum up all the positions of each point
        set_sum += meanSet[i][j]
        pass
      # update mu to the average of each point in mu's cluster
      if len(meanSet[i]) != 0:
        mu[i] = set_sum / len(meanSet[i])
        pass
      pass
    return mu

  def calculate_total_error(self, meanSet, mu):
    mserror = 0
    N = 0
    for i in range(self.k):
      for j in range(len(meanSet[i])):
        error = np.linalg.norm(meanSet[i][j] - mu[i])
        mserror += error * error
        N += 1
        pass
      pass
    return mserror / N
  
  pass



In [110]:
# Lloyd's Algorithm- application

# Set up X as independent dataframe with features vector
clusterDF = train_set
featureLabels = copy.deepcopy(features)
featureLabels.remove(target)

# Let K increase logarithmically with N.
# This will help optimize clustering and create good representatives.

train_N = clusterDF.shape[0]
K = 10 * int(math.floor(math.log(train_N)))

lloyd = LloydMeans(K, clusterDF, featureLabels)
mu = lloyd.assign_means(num_iterations = 50, num_trials = 10)

MSE for trial 0 : 0.254862599608
MSE for trial 1 : 0.104558737557
MSE for trial 2 : 0.213666324099
MSE for trial 3 : 0.105426345453
MSE for trial 4 : 0.0776927485398
MSE for trial 5 : 0.0631433716969
MSE for trial 6 : 0.105884978989
MSE for trial 7 : 0.0776927485398
MSE for trial 8 : 0.104558737557
MSE for trial 9 : 0.254862599608
Best error is 0.0631433716969


### Step 2: Defining the Radial Basis Function

Here is the function again, just to remind myself, and because LaTEX is awesome.

\begin{equation}
\large
y = \sum_{k=1}^K w_ke^{-\lambda \| \mathbf{x} - \mathbf{\mu_k} \| ^ 2}
\end{equation}


In [0]:
def _predict (mu, w, x, precision_rate):
#   Sum over mu
  sum = 0
  for i in range(mu.shape[0]):
#     distance between point x and mu
    dist = np.linalg.norm(x[i] - mu[i])
#   square distance, multiply by lambda
    exponent = (- precision_rate) * dist * dist
#   multiply exponent by weight, add to total
    sum += (w[i] * math.exp(exponent))
    pass
  return sum


This is a good start, but we have to visualize this as a matrix instead of a function before we can solve for $ \mathbf{w} $

We're going to do this by creating the following matrix:

$\mathbf{A_{1,K}} = 
\begin{pmatrix} 
e^{-\lambda \| \mathbf{x} - \mathbf{\mu_1} \| ^ 2} & \cdots & e^{-\lambda \| \mathbf{x} - \mathbf{\mu_K} \| ^ 2}
\end{pmatrix}$

This way, the new equation is:
$
\begin{equation}
\begin{bmatrix} 
e^{-\lambda \| \mathbf{x} - \mathbf{\mu_1} \| ^ 2} & \cdots & e^{-\lambda \| \mathbf{x} - \mathbf{\mu_K} \| ^ 2}
\end{bmatrix}
\begin{bmatrix}
w_1 \\ \vdots \\ w_K
\end{bmatrix} = y
\end{equation}
$

or

$\begin{equation} \mathbf{Aw} = y \end{equation}$

In [0]:
def predict (mu, w, x, precision_rate):
  A = np.zeros((1, mu.shape[0]))
  for i in range(mu.shape[0]):
#   distance between point x and mu
    dist = np.linalg.norm(x - mu[i])
#   square distance, multiply by lambda
    exponent = (- precision_rate) * dist * dist
    A[0][i] = math.exp(exponent)
    pass
  return A.dot(w)

### Step 3: Training.


This is going to be the most fun part.
Extend our previous matrix $ \mathbf{A} $ to all N points in our dataset, creating a K by N matrix.

$\mathbf{A_{N,K}} = 
\begin{bmatrix} 
e^{-\lambda \| \mathbf{x_1} - \mathbf{\mu_1} \| ^ 2} & 
e^{-\lambda \| \mathbf{x_1} - \mathbf{\mu_2} \| ^ 2} & \cdots & 
e^{-\lambda \| \mathbf{x_1} - \mathbf{\mu_K} \| ^ 2} 
\\
e^{-\lambda \| \mathbf{x_2} - \mathbf{\mu_1} \| ^ 2} & 
e^{-\lambda \| \mathbf{x_2} - \mathbf{\mu_2} \| ^ 2} & \cdots &
e^{-\lambda \| \mathbf{x_2} - \mathbf{\mu_K} \| ^ 2}
\\
\vdots & \vdots & \ddots & \vdots
\\
e^{-\lambda \| \mathbf{x_N} - \mathbf{\mu_1} \| ^ 2} &
e^{-\lambda \| \mathbf{x_N} - \mathbf{\mu_2} \| ^ 2} &\cdots &
e^{-\lambda \| \mathbf{x_N} - \mathbf{\mu_K} \| ^ 2} 
\end{bmatrix}$

Now,

$
\begin{equation}
\begin{bmatrix} 
e^{-\lambda \| \mathbf{x_1} - \mathbf{\mu_1} \| ^ 2} & 
e^{-\lambda \| \mathbf{x_1} - \mathbf{\mu_2} \| ^ 2} & \cdots & 
e^{-\lambda \| \mathbf{x_1} - \mathbf{\mu_K} \| ^ 2} 
\\
e^{-\lambda \| \mathbf{x_2} - \mathbf{\mu_1} \| ^ 2} & 
e^{-\lambda \| \mathbf{x_2} - \mathbf{\mu_2} \| ^ 2} & \cdots &
e^{-\lambda \| \mathbf{x_2} - \mathbf{\mu_K} \| ^ 2}
\\
\vdots & \vdots & \ddots & \vdots
\\
e^{-\lambda \| \mathbf{x_N} - \mathbf{\mu_1} \| ^ 2} &
e^{-\lambda \| \mathbf{x_N} - \mathbf{\mu_2} \| ^ 2} &\cdots &
e^{-\lambda \| \mathbf{x_N} - \mathbf{\mu_K} \| ^ 2} 
\end{bmatrix}
\begin{bmatrix}
w_1 \\ w_2 \\ \vdots \\ w_K
\end{bmatrix}
=
\begin{bmatrix}
y_1 \\ y_2 \\ \vdots \\ y_N
\end{bmatrix}
\end{equation}$

or,

$ \mathbf{Aw = y} $

So, to solve for $ \mathbf{w} $ given a reasonable lambda we use the old Math 415 least squares regression equation

$\begin{equation}
\mathbf{w = (A^TA)^{-1}A^Ty}
\end{equation}
$

We're also going to write a loss function to look for a good MSE

In [0]:
def train (X, Y, precision_rate, mu):
  A = np.zeros((X.shape[0], mu.shape[0]))
#   iterate over X
  for i in range(X.shape[0]):
#     iterate over mu
    for j in range(mu.shape[0]):
#     create vectors for x and mu
      _x = np.transpose([X[i]])
      _mu = np.transpose([mu[j]])
#     Take the distance between point x and mu
      dist = np.linalg.norm(_x - _mu)
  #   square distance, multiply by lambda
      exponent = (- precision_rate) * dist * dist
      A[i][j] = math.exp(exponent)
      pass
    pass
# Now we have A, let's get the appropriate transformation for y
  trans = np.transpose(A).dot(A).dot(np.transpose(A))
  return trans.dot(Y)
  
  
def get_loss(X, Y, precision_rate, mu, w):
  mse = 0
  count = 0
  for i in range(X.shape[0]):
    predicted = predict(mu, w, X[i], precision_rate)
    err = Y[i] - predicted
    mse += err * err
    count += 1
    pass
  res = mse / count
  return res

### Step 4: Application
I've now written more latex than code and tested none of the code. Let's see if anything happens.
I'm going to put everything in a for loop that tries lambdas ranging between 0.1 and 2.

In [125]:
step = 0.0625
precision_rate = 84.5
X = train_set.drop(target, axis=1).values
Y = train_set[[target]].values

test_X = test_set.drop(target, axis=1).values
test_Y = test_set[[target]].values

best_precision = precision_rate
best_error = float('inf')

for i in range(5):
#   Train for given precision rate
  w = train(X, Y, precision_rate, mu)
#   Get the error
  current_error = get_loss(test_X, test_Y, precision_rate, mu, w)
  print('MSE for lambda {} was {}'.format(precision_rate, current_error))
  
#   Replace the best error and rate if a better set is found
  if current_error < best_error:
    best_error = current_error
    best_precision = precision_rate
    pass
  precision_rate += step
  pass

print('Best MSE was {} with precision rate {}'.format(best_error, best_precision))
  

MSE for lambda 84.5 was [[41989.73420287]]
MSE for lambda 84.5625 was [[41989.17703309]]
MSE for lambda 84.625 was [[41988.69360827]]
MSE for lambda 84.6875 was [[41988.28299636]]
MSE for lambda 84.75 was [[41987.9442752]]
Best MSE was [[41987.9442752]] with precision rate 84.75


### Conclusion:
...Fuck.

...in case the code block's output isn't visible, this is what it looks like:


```

MSE for lambda 84.5 was [[41989.73420287]]
MSE for lambda 84.5625 was [[41989.17703309]]
MSE for lambda 84.625 was [[41988.69360827]]
MSE for lambda 84.6875 was [[41988.28299636]]
MSE for lambda 84.75 was [[41987.9442752]]
Best MSE was [[41987.9442752]] with precision rate 84.75
```



#### Possible issues:
 - N too small.
 - K too small.
 - Features are too few.
 - lambda uniform across means.
 - model not complex enough.
 - Some stupid mistake somewhere scales values wrong. You know, by a factor of $10^5$.
 
#### Possible Solutions:
  - Fuck this dataset. This dataset is horrible. 400 points, 7 features.
  - Try to find/ implement variable lambda, and use iterative algorithm to regress w and lambda
  - Implement RBF Network with multiple layers. This would be ridiculous in terms of debugging and time commitment.

## Kush's Network

The rationale behind creating a neural network with the NumofSHSATTestTakers is to determine the most probable schools which would have more test takers and then predict other schools' probable students. Following which we can rank each school depending on how diverse they are

In [0]:
# Filter the dataset to remove any schools without any SHSAT Test takers

features = ['TestTakersPercentage','ELA8thGrade4s','Math8thGrade4s','Percent of Students Chronically Absent','Average Math Proficiency','Economic Need Index','District','Average ELA Proficiency']

nytCombinedFiltered = nytCombined.query("NumSHSATTestTakers !=0")

Math8thGrade4s = pd.Series(nytCombinedFiltered[ 'Grade 8 Math 4s - All Students']/(nytCombinedFiltered[ 'Grade 8 Math - All Students Tested']*1.0))
ELA8thGrade4s = pd.Series(nytCombinedFiltered[ 'Grade 8 ELA 4s - All Students']/(nytCombinedFiltered[ 'Grade 8 ELA - All Students Tested']*1.0))
TestTakersPercentage = pd.Series(nytCombinedFiltered['NumSHSATTestTakers']/(nytCombinedFiltered[ 'Grade 8 Math - All Students Tested']*1.0))

nytCombinedFiltered = nytCombinedFiltered.assign(Math8thGrade4s = Math8thGrade4s)
nytCombinedFiltered = nytCombinedFiltered.assign(ELA8thGrade4s = ELA8thGrade4s)
nytCombinedFiltered = nytCombinedFiltered.assign(TestTakersPercentage = TestTakersPercentage)


def removePercent(df,columnName):
    df[columnName] = df[columnName].str.rstrip('%').astype('float') / 100.0
    features.append(columnName)
    return

removePercent(nytCombinedFiltered,'Student Attendance Rate')
removePercent(nytCombinedFiltered,'Percent of Students Chronically Absent')

nytCombinedFiltered.fillna(0,inplace = True)
nytCombinedFiltered = nytCombinedFiltered.replace([np.inf, -np.inf], np.nan)
nytCombinedFiltered = nytCombinedFiltered.dropna(subset=['TestTakersPercentage'])
# nytCombinedFiltered.fillna(1,inplace = True)

In [0]:
numFeatures = len(features)
print(numFeatures)

10


In [0]:
# nytCombinedFiltered['Student Attendance Rate']

In [0]:
# We split the Combined and filtered dataset to training and testing datasets, 60 each
datasetSplit = 150

nytCombinedFilteredTraining = nytCombinedFiltered[0:datasetSplit]
nytCombinedFilteredTesting = nytCombinedFiltered[datasetSplit:]

# Only pick out the columns that we want, ie: the ones with numeric values, we can still access the exact school name cause of the index

nytTraining = nytCombinedFilteredTraining.filter(features)
nytTesting = nytCombinedFilteredTesting.filter(features)


In [0]:
# list(nytTesting)



In [0]:
# Scale the inputs and outputs to a suitable small range
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
scaled_training = scaler.fit_transform(nytTraining)
scaled_testing = scaler.transform(nytTesting)

scaled_training_df = pd.DataFrame(scaled_training, columns= nytTraining.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing, columns=nytTesting.columns.values)

scaled_training_df.to_csv("kaggle-data/training_scaled.csv", index=False)
scaled_testing_df.to_csv("kaggle-data/testing_scaled.csv", index=False)

print("Note: Number of SHSAT Taker values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[0], scaler.min_[0]))

Note: Number of SHSAT Taker values were scaled by multiplying by 0.1331344397 and adding -0.006340


In [0]:
#This is the training of the model

#This didnt work trying not the scaled values

X = scaled_training_df.drop('TestTakersPercentage', axis=1).values

# Using the output as a percentage

Y = scaled_training_df[['TestTakersPercentage']].values


# X = nytTraining.drop('NumSHSATTestTakers', axis=1).values
# Y = nytTraining[['NumSHSATTestTakers']].values


In [0]:
# Define the model
model = Sequential()
activationFuntion = 'relu'
model.add(Dense(50, input_dim=numFeatures-1, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
# model.compile(loss="mean_squared_error", optimizer="adam")

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','accuracy'])


#Train
model.fit(
    X,
    Y,
    epochs=50,
    shuffle=True,
    verbose=2
)

X_test = scaled_testing_df.drop('TestTakersPercentage', axis=1).values
Y_test = scaled_testing_df[['TestTakersPercentage']].values

test_error_rate = model.evaluate(X_test, Y_test, verbose=0)

testErrorArray.append(test_error_rate)
print("The mean squared error (MSE) for the test data set is: {}".format(test_error_rate))
print("Here: \n%s: %.5f%%" % (model.metrics_names[2], test_error_rate[2]))

model.save('savedModel.h5')
print("Model Saved to Folder")

Epoch 1/50
 - 1s - loss: 0.0138 - mean_absolute_error: 0.0621 - acc: 0.0067
Epoch 2/50
 - 0s - loss: 0.0105 - mean_absolute_error: 0.0580 - acc: 0.0067
Epoch 3/50
 - 0s - loss: 0.0096 - mean_absolute_error: 0.0498 - acc: 0.0067
Epoch 4/50
 - 0s - loss: 0.0089 - mean_absolute_error: 0.0432 - acc: 0.0067
Epoch 5/50
 - 0s - loss: 0.0082 - mean_absolute_error: 0.0470 - acc: 0.0067
Epoch 6/50
 - 0s - loss: 0.0076 - mean_absolute_error: 0.0410 - acc: 0.0067
Epoch 7/50
 - 0s - loss: 0.0072 - mean_absolute_error: 0.0443 - acc: 0.0067
Epoch 8/50
 - 0s - loss: 0.0067 - mean_absolute_error: 0.0399 - acc: 0.0067
Epoch 9/50
 - 0s - loss: 0.0065 - mean_absolute_error: 0.0406 - acc: 0.0067
Epoch 10/50
 - 0s - loss: 0.0063 - mean_absolute_error: 0.0422 - acc: 0.0067
Epoch 11/50
 - 0s - loss: 0.0062 - mean_absolute_error: 0.0414 - acc: 0.0067
Epoch 12/50
 - 0s - loss: 0.0067 - mean_absolute_error: 0.0412 - acc: 0.0067
Epoch 13/50
 - 0s - loss: 0.0065 - mean_absolute_error: 0.0520 - acc: 0.0067
Epoch 14

In [0]:
#Loading the saved model and then using it:

model = load_model('savedModel.h5')

# prediction = model.predict(X)

In [0]:
X = pd.read_csv('kaggle-data/training_scaled.csv')


X.head()

X = X.loc[0:10]

before = X[0:20]['TestTakersPercentage']
X = X.drop('TestTakersPercentage',axis = 1)
prediction = model.predict(X)
# prediction = prediction + scaler.min_[0]
prediction = prediction / scaler.scale_[0]


print(before,prediction)



0     0.092235
1     0.115882
2     1.000000
3     0.226646
4     0.510349
5     0.102989
6     0.123342
7     0.134662
8     0.087131
9     0.075038
10    0.079187
Name: TestTakersPercentage, dtype: float64 [[0.8120292 ]
 [0.77867067]
 [4.939108  ]
 [1.6150904 ]
 [4.0077496 ]
 [0.69094396]
 [0.8725062 ]
 [1.1068518 ]
 [0.51014334]
 [0.7604499 ]
 [0.5523411 ]]
